<a href="https://colab.research.google.com/github/fajrunwm/ledf/blob/main/LEDF_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEDF

Localised Event Demand Forecasting Model Building using Random Forest Classifier

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

# Load the dataset
file_path = "/content/drive/My Drive/Colab Notebooks/LEDF/data/eventbrite/combined_eventbrite_data.csv"
data = pd.read_csv(file_path)

# Preview the data
data.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 144 columns):
 #    Column                                                                                  Dtype  
---   ------                                                                                  -----  
 0    events__results__tags___type                                                            float64
 1    current_user_id                                                                         float64
 2    events__pagination__object_count                                                        float64
 3    events__pagination__page_count                                                          float64
 4    events__pagination__page_number                                                         float64
 5    events__pagination__page_size                                                           float64
 6    events__pagination__continuation                                        

In [ ]:
# # Select relevant columns by their names
# selected_columns = [
#     "events__results__name",
#     # "events__results__summary",
#     # "events__results__full_description",
#     "events__results__tags__display_name",
#     "events__results__start_date",
#     "events__results__end_date",
#     "events__results__start_time",
#     "events__results__end_time",
#     "events__results__primary_venue__name",
#     "events__results__primary_venue__address__city",
#     "events__results__primary_venue__address__country",
#     "events__results__primary_venue__address__latitude",
#     "events__results__primary_venue__address__longitude",
#     "events__results__ticket_availability__is_free",
#     "events__results__ticket_availability__minimum_ticket_price__value",
#     "events__results__ticket_availability__maximum_ticket_price__value",
#     "events__results__ticket_availability__has_available_tickets",
#     "events__results__primary_organizer__name",
#     # "events__results__primary_organizer__num_upcoming_events",
#     # "events__results__saves__saved_by_you",
#     "events__results__primary_organizer__num_followers",
#     "events__results__is_online_event",
#     # "events__results__is_cancelled",
#     "events__results__language",
#     "events__results__urgency_signals__messages__001"
# ]

# # Now you can use the selected_columns list with pandas .loc[] or bracket notation
# filtered_data = data[selected_columns]  # OR filtered_data = data.loc[:, selected_columns]
# filtered_data.head()

# # Rename columns
# new_column_names = {
#     "events__results__name": "event_name",
#     # "events__results__summary": "summary",
#     # "events__results__full_description": "full_description",
#     "events__results__tags__display_name": "tags",
#     "events__results__start_date": "start_date",
#     "events__results__end_date": "end_date",
#     "events__results__start_time": "start_time",
#     "events__results__end_time": "end_time",
#     "events__results__primary_venue__name": "venue_name",
#     "events__results__primary_venue__address__city": "city",
#     "events__results__primary_venue__address__country": "country",
#     "events__results__primary_venue__address__latitude": "latitude",
#     "events__results__primary_venue__address__longitude": "longitude",
#     "events__results__ticket_availability__is_free": "is_free",
#     "events__results__ticket_availability__minimum_ticket_price__value": "min_ticket_price",
#     "events__results__ticket_availability__maximum_ticket_price__value": "max_ticket_price",
#     "events__results__ticket_availability__has_available_tickets": "has_available_tickets",
#     "events__results__primary_organizer__name": "organizer_name",
#     # "events__results__primary_organizer__num_upcoming_events": "num_upcoming_events",
#     # "events__results__saves__saved_by_you": "saved_by_you",
#     "events__results__primary_organizer__num_followers": "organizer_followers",
#     "events__results__is_online_event": "is_online_event",
#     # "events__results__is_cancelled": "is_cancelled",
#     "events__results__language": "language",
#     "events__results__urgency_signals__messages__001": "urgency_message"
# }

# filtered_data = filtered_data.rename(columns=new_column_names)

In [3]:
filtered_data = data.iloc[:, [34, 46, 83, 77, 47, 67, 71, 73, 74, 75, 76, 80, 90, 95, 128, 130]]

In [4]:
# Define a dictionary to map old column names to new names
new_column_names = {
    "events__results__id": "event_id",
    "events__results__start_date": "start_date",
    "events__results__start_time": "start_time",
    "events__results__end_date": "end_date",
    "events__results__end_time": "end_time",
    "events__results__ticket_availability__maximum_ticket_price__value": "max_ticket_price",
    "events__results__ticket_availability__minimum_ticket_price__value": "min_ticket_price",
    "events__results__ticket_availability__is_free": "is_free",
    "events__results__ticket_availability__has_bogo_tickets": "has_bogo_tickets",
    "events__results__ticket_availability__has_available_tickets": "has_available_tickets",
    "events__results__ticket_availability__is_sold_out": "is_sold_out",
    "events__results__tags__display_name": "tags",
    "events__results__primary_venue__address__longitude": "longitude",
    "events__results__primary_venue__address__latitude": "latitude",
    "events__results__hide_start_date": "hide_start_date",
    "events__results__is_online_event": "is_online_event",
}

# Rename the columns
filtered_data.rename(columns=new_column_names, inplace=True)

<ipython-input-4-98f0f8d7929e>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.rename(columns=new_column_names, inplace=True)


In [5]:
filtered_data.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   event_id               180 non-null    float64
 1   start_date             180 non-null    object 
 2   start_time             180 non-null    object 
 3   end_date               180 non-null    object 
 4   end_time               180 non-null    object 
 5   max_ticket_price       180 non-null    float64
 6   min_ticket_price       180 non-null    float64
 7   is_free                180 non-null    bool   
 8   has_bogo_tickets       180 non-null    bool   
 9   has_available_tickets  180 non-null    bool   
 10  is_sold_out            180 non-null    bool   
 11  tags                   179 non-null    object 
 12  longitude              180 non-null    float64
 13  latitude               180 non-null    float64
 14  hide_start_date        180 non-null    bool   
 15  is_onl

In [6]:
 # Show random samples
filtered_data.sample(5)

,event_id,start_date,start_time,end_date,end_time,max_ticket_price,min_ticket_price,is_free,has_bogo_tickets,has_available_tickets,is_sold_out,tags,longitude,latitude,hide_start_date,is_online_event
105,1.131103e+12,2025-01-22,09:00,2025-01-22,11:00,5873.0,5873.0,False,False,False,True,Baby,103.795639,1.310792,False,False
73,1.146809e+12,2025-01-25,22:00,2025-01-26,03:30,4826.0,1682.0,False,False,True,False,Music,103.849481,1.282171,False,False
128,1.086603e+12,2025-01-18,09:00,2025-01-18,12:30,22743.0,13522.0,False,False,True,False,Medical,103.849800,1.291477,False,False
107,1.135202e+12,2025-01-25,22:00,2025-01-26,04:00,2731.0,1682.0,False,False,True,False,Music,103.848346,1.289138,False,False
174,1.100016e+12,2025-01-17,17:30,2025-01-17,22:00,661.0,661.0,False,False,True,False,High Tech,103.848238,1.303942,False,False


In [7]:
# Check for missing values in the dataset
# filtered_data.isnull().sum()

# Fill missing values if appropriate, e.g., fill with '0' or 'mean' for numerical values:
# filtered_data['min_ticket_price'] = filtered_data['min_ticket_price'].fillna(0)
# filtered_data.head()

In [7]:
# Convert 'start_date', 'end_date', 'start_time', and 'end_time' to datetime format
filtered_data['start_date'] = pd.to_datetime(filtered_data['start_date'])
filtered_data['end_date'] = pd.to_datetime(filtered_data['end_date'])

# Combine 'end_date' and 'end_time' to create a new datetime column
filtered_data['start_time'] = pd.to_datetime(filtered_data['start_date'].dt.strftime('%Y-%m-%d') + ' ' + filtered_data['start_time'].astype(str), errors='coerce')
filtered_data['end_time'] = pd.to_datetime(filtered_data['end_date'].dt.strftime('%Y-%m-%d') + ' ' + filtered_data['end_time'].astype(str), errors='coerce')

<ipython-input-7-03382d98df58>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['start_date'] = pd.to_datetime(filtered_data['start_date'])
<ipython-input-7-03382d98df58>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['end_date'] = pd.to_datetime(filtered_data['end_date'])
<ipython-input-7-03382d98df58>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [8]:
# Create additional features from datetime columns with minutes in decimals
filtered_data['start_hour'] = filtered_data['start_time'].dt.hour + (filtered_data['start_time'].dt.minute / 60.0)
filtered_data['end_hour'] = filtered_data['end_time'].dt.hour + (filtered_data['end_time'].dt.minute / 60.0)

# start_weekday remains the same as it's based on the day of the week, not the time
# filtered_data['start_weekday'] = filtered_data['start_date'].dt.weekday

<ipython-input-8-eb2553eb8dd8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['start_hour'] = filtered_data['start_time'].dt.hour + (filtered_data['start_time'].dt.minute / 60.0)
<ipython-input-8-eb2553eb8dd8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['end_hour'] = filtered_data['end_time'].dt.hour + (filtered_data['end_time'].dt.minute / 60.0)


In [9]:
# Create features based on the 'start_date' and 'end_date'
filtered_data['start_day_of_week'] = filtered_data['start_date'].dt.dayofweek  # Monday=0, Sunday=6
filtered_data['start_month'] = filtered_data['start_date'].dt.month
filtered_data['event_length_days'] = (filtered_data['end_date'] - filtered_data['start_date']).dt.days + 1

<ipython-input-9-1c8d97103153>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['start_day_of_week'] = filtered_data['start_date'].dt.dayofweek  # Monday=0, Sunday=6
<ipython-input-9-1c8d97103153>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['start_month'] = filtered_data['start_date'].dt.month
<ipython-input-9-1c8d97103153>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [10]:
# Calculate event duration in hour (optional)
filtered_data['event_duration'] = (filtered_data['end_time'] - filtered_data['start_time']).dt.total_seconds() / 3600

<ipython-input-10-08543d10de45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['event_duration'] = (filtered_data['end_time'] - filtered_data['start_time']).dt.total_seconds() / 3600


In [13]:
# # Drop rows with missing tags and organizer_name
# filtered_data = filtered_data.dropna(subset=['tags', 'organizer_name', 'organizer_followers'])

# # Fill missing urgency_message with empty string
# filtered_data['urgency_message'] = filtered_data['urgency_message'].fillna('')

# Fill missing organizer_followers with the median value
# median_followers = filtered_data['organizer_followers'].median()
# filtered_data['organizer_followers'] = filtered_data['organizer_followers'].fillna(median_followers)

In [12]:
# Drop original datetime columns after creating numerical features
filtered_data = filtered_data.drop(columns=['start_date', 'end_date', 'start_time', 'end_time'])

## Inspect

In [14]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   event_id               180 non-null    float64
 1   max_ticket_price       180 non-null    float64
 2   min_ticket_price       180 non-null    float64
 3   is_free                180 non-null    bool   
 4   has_bogo_tickets       180 non-null    bool   
 5   has_available_tickets  180 non-null    bool   
 6   is_sold_out            180 non-null    bool   
 7   tags                   179 non-null    object 
 8   longitude              180 non-null    float64
 9   latitude               180 non-null    float64
 10  hide_start_date        180 non-null    bool   
 11  is_online_event        180 non-null    bool   
 12  start_hour             180 non-null    float64
 13  end_hour               180 non-null    float64
 14  start_day_of_week      180 non-null    int32  
 15  start_

In [15]:
filtered_data.head()

,event_id,max_ticket_price,min_ticket_price,is_free,has_bogo_tickets,has_available_tickets,is_sold_out,tags,longitude,latitude,hide_start_date,is_online_event,start_hour,end_hour,start_day_of_week,start_month,event_length_days,event_duration
0,1.127364e+12,4313.0,4313.0,False,False,True,False,Music,103.849240,1.287790,False,False,22.5,4.0,5,1,2,5.5
1,1.131087e+12,5587.0,5587.0,False,False,True,False,Music,103.848346,1.289138,False,False,22.0,4.0,5,1,2,6.0
2,1.130412e+12,1500.0,1125.0,False,False,True,False,Classical,103.772120,1.302458,False,False,19.5,21.5,1,2,1,2.0
3,1.075581e+12,634.0,634.0,False,False,True,False,Science,103.847576,1.288155,False,False,18.0,21.0,3,1,1,3.0
4,1.092002e+12,6374.0,0.0,False,False,True,False,Historic,103.891708,1.332688,False,False,9.0,13.0,5,1,1,4.0


## Model Building (e.g., RandomForest, XGBoost, or another model)

In [16]:
# One-hot encode categorical columns
# dataset = pd.get_dummies(filtered_data, columns=['tags', 'language', 'city', 'country'], drop_first=True)
dataset = pd.get_dummies(filtered_data, columns=['tags'], drop_first=True)

In [17]:
# Keep 'tags' column available
dataset['tags_label'] = filtered_data['tags']
columnn = dataset.pop('tags_label')
dataset.insert(15, 'tags_label', columnn)

In [18]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the target variable
dataset['tags'] = label_encoder.fit_transform(dataset['tags_label'])

In [19]:
print(dataset[['tags_label', 'tags']].sort_values(by='tags'))

            tags_label  tags
158  After School Care     0
120        Alternative     1
5         Anime/Comics     2
58                Baby     3
105               Baby     3
..                 ...   ...
167               Yoga    52
28                Yoga    52
164               Yoga    52
159               Yoga    52
69                 NaN    53

[180 rows x 2 columns]


In [22]:
dataset=dataset.drop(columns=['tags_label'])

In [23]:
dataset.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 70 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   event_id                           180 non-null    float64
 1   max_ticket_price                   180 non-null    float64
 2   min_ticket_price                   180 non-null    float64
 3   is_free                            180 non-null    bool   
 4   has_bogo_tickets                   180 non-null    bool   
 5   has_available_tickets              180 non-null    bool   
 6   is_sold_out                        180 non-null    bool   
 7   longitude                          180 non-null    float64
 8   latitude                           180 non-null    float64
 9   hide_start_date                    180 non-null    bool   
 10  is_online_event                    180 non-null    bool   
 11  start_hour                         180 non-null    float64

In [24]:
dataset.sample(5)

,event_id,max_ticket_price,min_ticket_price,is_free,has_bogo_tickets,has_available_tickets,is_sold_out,longitude,latitude,hide_start_date,...,tags_R&B,tags_Rock,tags_Sales & Marketing,tags_Science,tags_Spirits,tags_Startups & Small Business,tags_Top 40,tags_Weightlifting,tags_Yoga,tags
123,1.107613e+12,41096.0,20551.0,False,False,True,False,103.859106,1.283758,False,...,False,False,False,False,False,True,False,False,False,49
137,1.103528e+12,36693.0,36693.0,False,False,True,False,103.845839,1.276065,False,...,False,False,False,False,False,False,False,False,False,41
113,1.137373e+12,5000.0,2544.0,False,False,True,False,103.847479,1.288239,False,...,False,False,False,False,False,False,True,False,False,50
13,1.125925e+12,2880.0,2280.0,False,False,True,False,103.898359,1.326891,False,...,False,False,False,False,False,False,False,False,False,42
58,1.131067e+12,4826.0,4826.0,False,False,False,True,103.848907,1.299427,False,...,False,False,False,False,False,False,False,False,False,3


In [38]:
# from sklearn.preprocessing import MinMaxScaler

# # Select numeric columns to normalize
# numeric_columns = ['latitude', 'longitude', 'min_ticket_price', 'max_ticket_price', 'organizer_followers', 'event_duration']
# scaler = MinMaxScaler()

# # Normalize the data and replace in the DataFrame
# dataset[numeric_columns] = scaler.fit_transform(dataset[numeric_columns])

In [25]:
# Train-Test Split
from sklearn.model_selection import train_test_split

# Define features and target
# X = dataset.drop(columns=['tags', 'event_name', 'organizer_name', 'urgency_message', 'venue_name'])  # Exclude unused fields
X = dataset.drop(columns=['tags'])  # Exclude unused fields
y = dataset['tags']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train-Test Split
from sklearn.model_selection import train_test_split

# Define features and target
X = dataset.drop(columns=['tags', 'event_name', 'organizer_name', 'urgency_message', 'venue_name'])  # Exclude unused fields
y = dataset['tags']

# Check for any remaining datetime columns in X
remaining_datetime_cols = X.select_dtypes(include=['datetime']).columns

# Drop remaining datetime columns if any
if len(remaining_datetime_cols) > 0:
    X = X.drop(columns=remaining_datetime_cols)
    print(f"Dropped datetime columns: {remaining_datetime_cols.tolist()}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train-Test Split
from sklearn.model_selection import train_test_split

# Define features and target
X = dataset.drop(columns=['tags', 'event_name', 'organizer_name', 'urgency_message', 'venue_name'])  # Exclude unused fields
y = dataset['tags']

# Check for any remaining datetime columns in X
remaining_datetime_cols = X.select_dtypes(include=['datetime']).columns

# Drop remaining datetime columns if any
if len(remaining_datetime_cols) > 0:
    X = X.drop(columns=remaining_datetime_cols)
    print(f"Dropped datetime columns: {remaining_datetime_cols.tolist()}")

# Convert 'start_date' and 'end_date' to numerical features (e.g., ordinal) before splitting
for col in ['start_date', 'end_date', 'start_time', 'end_time']:  # Include all datetime columns
    if col in X.columns:
        X[col] = X[col].astype(int) // 10**9  # Convert to Unix timestamp (seconds)


# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Dropped datetime columns: ['start_date', 'end_date', 'start_time', 'end_time']


In [26]:
X.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 69 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   event_id                           180 non-null    float64
 1   max_ticket_price                   180 non-null    float64
 2   min_ticket_price                   180 non-null    float64
 3   is_free                            180 non-null    bool   
 4   has_bogo_tickets                   180 non-null    bool   
 5   has_available_tickets              180 non-null    bool   
 6   is_sold_out                        180 non-null    bool   
 7   longitude                          180 non-null    float64
 8   latitude                           180 non-null    float64
 9   hide_start_date                    180 non-null    bool   
 10  is_online_event                    180 non-null    bool   
 11  start_hour                         180 non-null    float64

In [27]:
X.sample(5)

,event_id,max_ticket_price,min_ticket_price,is_free,has_bogo_tickets,has_available_tickets,is_sold_out,longitude,latitude,hide_start_date,...,tags_Personal health,tags_R&B,tags_Rock,tags_Sales & Marketing,tags_Science,tags_Spirits,tags_Startups & Small Business,tags_Top 40,tags_Weightlifting,tags_Yoga
38,1.124252e+12,157279.0,3778.0,False,False,True,False,103.843249,1.279485,False,...,False,False,False,False,False,False,False,False,False,False
111,1.132238e+12,3778.0,2731.0,False,False,True,False,103.848420,1.278646,False,...,False,False,False,False,False,False,False,False,False,False
67,1.130664e+12,1500.0,1125.0,False,False,True,False,103.851562,1.288279,False,...,False,False,False,False,False,False,False,False,False,False
65,1.111283e+12,4500.0,4500.0,False,False,True,False,103.853918,1.285641,False,...,False,False,False,False,False,False,False,False,False,False
27,1.117157e+12,2000.0,2000.0,False,False,True,False,103.850039,1.279695,False,...,False,False,False,False,False,False,False,False,False,False


In [28]:
y.info(all)

<class 'pandas.core.series.Series'>
RangeIndex: 180 entries, 0 to 179
Series name: tags
Non-Null Count  Dtype
--------------  -----
180 non-null    int64
dtypes: int64(1)
memory usage: 1.5 KB


In [29]:
y.sample(5)

,tags
78,12
40,43
176,26
71,14
80,19


In [30]:
# Train a Random Forest Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Initialize and train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predictions and Evaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         1
           8       1.00      1.00      1.00         2
          10       0.00      0.00      0.00         1
          11       0.67      1.00      0.80         2
          12       1.00      1.00      1.00         3
          15       0.00      0.00      0.00         0
          17       1.00      1.00      1.00         1
          22       0.00      0.00      0.00         1
          24       0.00      0.00      0.00         0
          26       0.00      0.00      0.00         1
          29       0.00      0.00      0.00         1
          32       0.00      0.00      0.00         1
          33       1.00      1.00      1.00         1
          34       0.00      0.00      0.00         0
          35       1.00      1.00      1.00         1
          36       0.50    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_